In [1]:
import torch
import torch.nn as nn

# Implemented by myself
from config import *
from data_processer import CSCDataset, split_torch_dataset
from models import CombineBertModel, DecoderBaseRNN, DecoderTransformer, Trainer
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Tokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)

#### DataSet

In [3]:
train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)
test_dataset = CSCDataset([SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer)

preprocessing sighan dataset: 2339it [00:00, 965978.44it/s]
preprocessing sighan dataset: 100%|██████████| 2339/2339 [00:00<00:00, 1584635.29it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 888877.97it/s]
preprocessing sighan dataset: 100%|██████████| 3437/3437 [00:00<00:00, 1370846.60it/s]

共3437句，共170330字，最长的句子有258字


In [4]:
# split data
train_data, dev_data = split_torch_dataset(train_dataset, 0.3)

train_data_loader = DataLoader(train_data, num_workers=4, shuffle=True, batch_size=16)
dev_data_loader = DataLoader(dev_data, num_workers=4, shuffle=True, batch_size=16)
test_data_loader = DataLoader(test_dataset, num_workers=4, shuffle=True, batch_size=32)

#### Config
most config from config.py

In [5]:
# epochs = 35

#### BERT + LSTM

In [6]:
# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

encoder_model = BertModel.from_pretrained(checkpoint)
decoder_model = DecoderBaseRNN(
    model=nn.LSTM,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)
model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

optimizer = AdamW(model.parameters(), lr=learning_rate)
trainer = Trainer(model=model, tokenizer=tokenizer, optimizer=optimizer)

In [7]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.93it/s, batches loss=5.807]

5.756522612138228 {'over_corr': 20612, 'total_err': 925, 'true_corr': tensor(17, device='cuda:0')} {'over_corr': 701, 'total_err': 701, 'true_corr': 0}



dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.93it/s, batches loss=5.641]

5.63110322302038 {'over_corr': 20511, 'total_err': 925, 'true_corr': tensor(1, device='cuda:0')} {'over_corr': 701, 'total_err': 701, 'true_corr': 0}



dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.95it/s, batches loss=5.664]

...

0.4357498958706856 {'over_corr': 335, 'total_err': 925, 'true_corr': tensor(312, device='cuda:0')} {'over_corr': 215, 'total_err': 701, 'true_corr': 166}



dev Epoch:1/1: 100%|██████████| 44/44 [00:08<00:00,  5.34it/s, batches loss=0.292]

0.42910330403934827 {'over_corr': 317, 'total_err': 925, 'true_corr': tensor(310, device='cuda:0')} {'over_corr': 200, 'total_err': 701, 'true_corr': 171}



dev Epoch:1/1: 100%|██████████| 108/108 [00:38<00:00,  2.84it/s, batches loss=0.404]

0.5043595328375146 {'over_corr': 3832, 'total_err': 5278, 'true_corr': tensor(1187, device='cuda:0')} {'over_corr': 1710, 'total_err': 3436, 'true_corr': 378}


In [42]:
model = torch.load(save_path)

In [44]:
# import time
# from utils import cal_err

# def split_lines(i, length=20):
#     print(f"第{i}句".center(length, '-'))

# def effectiveness_of_beam(model, test_data, beam_width):
#     begin_time = time.time()
#     matrices = ["over_corr", "total_err", "true_corr"]
#     test_char_level = {key: 0 for key in matrices}
#     test_sent_level = {key: 0 for key in matrices}

#     for i, data in enumerate(test_data, 1):
#         best_score, best_sequence = -float('inf'), None
#         beam = model.generate_with_beam(beam_width, data)
#         for score, seq in beam:
#             if score > best_score:
#                 best_score = score
#                 best_sequence = seq
#         split_lines(i)
#         input_ids = data["input_ids"]
#         labels = data["labels"]
#         attention_mask = data["attention_mask"]

#         raw_model_output = model(input_ids.resize(1, len(input_ids)).to("cuda"),
#                   attention_mask.resize(1, len(input_ids)).to("cuda"))

#         length = len(best_sequence)
#         char_level, sent_level = cal_err(
#             input_ids[:length],
#             torch.tensor(best_sequence),
#             labels[:length],
#             length,
#         )
#         test_char_level = {
#             key: test_char_level[key] + v
#             for key, v in char_level.items()
#         }
#         test_sent_level = {
#             key: test_sent_level[key] + v
#             for key, v in sent_level.items()
#         }

#         print(f"origin sentence:  {tokenizer.decode(input_ids, skip_special_tokens=True)}")
#         print(f"correct sentence: {tokenizer.decode(labels, skip_special_tokens=True)}")
#         print(f"predict sentence: {tokenizer.decode(best_sequence, skip_special_tokens=True)}")
#         print("r-model sentence:", tokenizer.decode(raw_model_output.argmax(dim=-1).squeeze()[:length], skip_special_tokens=True))

#     end_time = time.time()
#     print(f"It cost total {end_time - begin_time} time")

# effectiveness_of_beam(model, test_dataset, 4)

In [41]:
save_path = "weights/BertLstm.pt"
model.save(save_path)

#### BERT + GRU

In [ ]:
# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

encoder_model = BertModel.from_pretrained(checkpoint)
decoder_model = DecoderBaseRNN(
    model=nn.GRU,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)
model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader, dev_data_loader)

In [ ]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

In [ ]:
effectiveness_of_beam(model, test_dataset, 4)

In [ ]:
save_path = "weights/BertGru.pt"
model.save(save_path)

#### BERT + Transformer

In [ ]:
nhead = 2
num_encoder_layers = 2
num_decoder_layers = 2

encoder_model = BertModel.from_pretrained(checkpoint)
decoder_model = DecoderTransformer(
    input_size=encoder_model.config.hidden_size,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
)
model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

train(model, tokenizer, train_data_loader, dev_data_loader)

In [ ]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

In [ ]:
save_path = "weights/BertTransformer.pt"
model.save(save_path)